# EJERCICIO DE DIFUSIÓN DE INFORMACIÓN
## Redes Sociales, Colaboración en Red

**Autores:**  
Martín Salinas Antón - martin.salinas@estudiante.uam.es  
Belén Vivas García - belen.vivas@estudiante.uam.es

In [1]:
import networkx as nx
import random
import numpy as np

### Lectura de Datos

En primer lugar, creamos el grafo mediante el fichero.

In [2]:
G = nx.read_graphml('data/juegoDtronos.graphml')

### Normalización de Pesos

El primer paso será normalizar los pesos de los vértices del grafo entre 0 y 1 para que el algoritmo los tome como probabilidades.

In [3]:
# Normalize weights into probabilities
weights = []
for edge in G.edges(data=True):
    # Get weight from graph and add it to the weight list
    weight = edge[2]['weight']
    weights.append(weight)

normalized_weights = []
# Get the maximum weight
max_weight = max(weights)
for w in weights:
    # Divide each weight by the maximum to normalize between 0 and 1
    w /= max_weight
    normalized_weights.append(w)

# Get a copy of the graph
G_norm = G
# Modify the weights
i = 0
for edge in G_norm.edges(data=True):
    edge[2]['weight'] = normalized_weights[i]
    i += 1

### Algoritmo

Implementamos los algoritmos necesarios para maximizar la influencia de la red. La influencia de la red se maximiza seleccionando un grupo de nodos, a los que llamaremos 'semilla', los cuales tienen una influencia suficiente para maximizar la propagación de información.  
  
La aproximación codiciosa (o greedy) selecciona los nodos con mayor influencia en función de la cantidad de vecinos, utilizando una función f que, en nuestro caso, selecciona el número de nodos que serán finalmente activados teniendo en cuenta los pesos (ahora probabilidades) de los vértices de los nodos.

In [4]:
# Maximizing the spread of cascades - greedy algorithm
def max_spread_cascades_greedy(G, k):

    # Set of initially activated nodes
    S = set()

    # While i =/= k
    for _ in range(k):
        v = None
        max_f = -1

        for node in G.nodes():
            if node not in S:
                # Cascade function taking S U {v}
                f = calculate_influence(G, S.union({node}))

                # Update maximized f
                if f > max_f:
                    v = node
                    max_f = f

        # Add chosen node to the set
        # S U {v}
        S.add(v)

    return S


# Gets the number of ultimately activated nodes
def calculate_influence(G, S):

    # Set of activated nodes
    A = S
    # Set of neighbors of nodes in S
    activable_neighbors = set()

    # Add neighbors to the set
    for node in S:
        activable_neighbors.update(G.neighbors(node))

    # While there are still activable neighbors
    while activable_neighbors:
        # Choose random neighbor
        node = random.choice(list(activable_neighbors))

        # Activation probability of the node
        probability = 1.0
        for n in G.neighbors(node):
            if n not in A:
                if n not in activable_neighbors:
                    # Take the node weight (now a probability)
                    weight = G[node][n]['weight']
                    probability *= (1 - weight)

        # Activation
        if np.random.rand() < probability:
            # Add activated node
            A.add(node)
            # Add neighbors of activated node
            for n in G.neighbors(node):
                if n not in A:
                    activable_neighbors.add(n)

        # Update neighbors
        activable_neighbors.remove(node)

    return len(A)

Establecemos k (budget) y ejecutamos el algoritmo para obtener la semilla.

In [5]:
# Set budget
k = 15
# Get seed
seed = max_spread_cascades_greedy(G_norm, k)

# Print results
print('Seed\n------------------\nBudget (k) = {}\n------------------'.format(k))
for s in seed:
    print(s)
print('-------------------')

Seed
------------------
Budget (k) = 15
------------------
Kevan-Lannister
Gyles-Rosby
Jalabhar-Xho
Walder-Frey
Brynden-Tully
Jaime-Lannister
Addam-Marbrand
Joffrey-Baratheon
Cersei-Lannister
Tyrion-Lannister
Lyle-Crakehall
Oberyn-Martell
Tywin-Lannister
Varys
Catelyn-Stark
-------------------
